In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta
import random

class DynamicScheduler:
    def __init__(self):
        self.timetable = {}
        self.user_preferences = {
            'weekday_sleep': 7,
            'weekend_sleep': 8,
            'self_study_time': 2,
            'extracurricular_time': 1,
            'gpa': 7.0,
            'calories_burned': 0
        }
        self.productivity_model = self.train_productivity_model()
        self.schedule = {}
        self.feedback_data = []

    def generate_stem_student_data(self):
        # Simulating 3 years of data for a STEM student
        dates = pd.date_range(start='2021-01-01', end='2023-12-31', freq='D')
        data = []
        
        for date in dates:
            for hour in range(24):
                productivity = 0
                if 9 <= hour <= 17:  # Higher productivity during typical study hours
                    productivity = np.random.normal(0.7, 0.1)
                elif 18 <= hour <= 22:  # Evening study session
                    productivity = np.random.normal(0.6, 0.1)
                else:
                    productivity = np.random.normal(0.3, 0.1)
                
                # Adjusting for weekends
                if date.dayofweek >= 5:
                    productivity *= 0.8
                
                # Adjusting for academic terms
                if date.month in [1, 2, 3, 4, 9, 10, 11, 12]:
                    productivity *= 1.2
                
                data.append({
                    'date': date,
                    'hour': hour,
                    'day_of_week': date.dayofweek,
                    'is_weekend': date.dayofweek >= 5,
                    'is_academic_term': date.month in [1, 2, 3, 4, 9, 10, 11, 12],
                    'productivity': max(0, min(1, productivity))  # Clamp between 0 and 1
                })
        
        return pd.DataFrame(data)

    def train_productivity_model(self):
        data = self.generate_stem_student_data()
        X = data[['hour', 'day_of_week', 'is_weekend', 'is_academic_term']]
        y = data['productivity']
        
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X, y)
        return model

    def get_timetable(self):
        days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        for day in days:
            if day in ['Saturday', 'Sunday']:
                print(f"\nEnter class timings for {day} (format: HH:MM-HH:MM, separate multiple entries with commas, or leave blank for no classes):")
            else:
                print(f"\nEnter class timings for {day} (format: HH:MM-HH:MM, separate multiple entries with commas):")
            timings = input().split(',')
            self.timetable[day] = [timing.strip() for timing in timings if timing.strip()]

    def get_user_preferences(self):
        print("\nEnter your preferences:")
        self.user_preferences['self_study_time'] = float(input("Desired self-study time (hours): "))
        self.user_preferences['extracurricular_time'] = float(input("Desired extracurricular/sports time (hours): "))
        self.user_preferences['gpa'] = float(input("Current GPA (0.0-10.0): "))

    def generate_schedule(self):
        days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        for day in days:
            day_schedule = []
            is_weekend = day in ['Saturday', 'Sunday']
            
            # Add fixed classes
            for slot in self.timetable[day]:
                start, end = slot.split('-')
                day_schedule.append((start, end, 'Class'))
            
            # Add sleep
            sleep_hours = self.user_preferences['weekend_sleep'] if is_weekend else self.user_preferences['weekday_sleep']
            day_schedule.append(('22:00', f'{(22 + sleep_hours) % 24:02d}:00', 'Sleep'))
            
            # Add meals
            breakfast_start = random.choice(['07:00', '07:30', '08:00', '08:30', '09:00'])
            day_schedule.append((breakfast_start, self.add_minutes(breakfast_start, 20), 'Breakfast'))
            
            lunch_start = random.choice(['12:30', '13:00', '13:30', '14:00'])
            day_schedule.append((lunch_start, self.add_minutes(lunch_start, 30), 'Lunch'))
            
            snack_start = random.choice(['16:30', '17:00', '17:30'])
            day_schedule.append((snack_start, self.add_minutes(snack_start, 15), 'Snack'))
            
            dinner_start = random.choice(['19:30', '20:00', '20:30', '21:00'])
            day_schedule.append((dinner_start, self.add_minutes(dinner_start, 30), 'Dinner'))
            
            # Sort the day_schedule
            day_schedule.sort(key=lambda x: datetime.strptime(x[0], '%H:%M'))
            
            # Find and allocate free slots
            free_slots = self.find_free_slots(day_schedule)
            self.allocate_activities(day_schedule, free_slots, is_weekend, day)
            
            self.schedule[day] = day_schedule

    def find_free_slots(self, day_schedule):
        free_slots = []
        for i in range(len(day_schedule)):
            current_end = datetime.strptime(day_schedule[i][1], '%H:%M')
            next_start = datetime.strptime(day_schedule[(i+1) % len(day_schedule)][0], '%H:%M')
            if next_start < current_end:
                next_start += timedelta(days=1)
            if (next_start - current_end).total_seconds() / 3600 >= 0.5:  # At least 30 minutes
                free_slots.append((current_end, next_start))
        return free_slots

    def allocate_activities(self, day_schedule, free_slots, is_weekend, day_of_week):
        study_time = self.user_preferences['self_study_time']
        extracurricular_time = self.user_preferences['extracurricular_time']
        
        # Adjust times based on GPA and calories
        if self.user_preferences['gpa'] < 7.0:
            study_time += 0.5
            extracurricular_time -= 0.25
        if self.user_preferences['calories_burned'] < 200:
            extracurricular_time += 0.25
        
        for start, end in free_slots:
            duration = (end - start).total_seconds() / 3600
            hour = start.hour
            day_index = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'].index(day_of_week)
            productivity = self.productivity_model.predict([[hour, day_index, is_weekend, True]])[0]  # Assuming it's always academic term
            
            if study_time > 0 and productivity > 0.6 and duration >= 0.5:
                activity_time = min(duration, study_time)
                day_schedule.append((start.strftime('%H:%M'), self.add_minutes(start.strftime('%H:%M'), int(activity_time * 60)), 'Self-Study'))
                study_time -= activity_time
            elif extracurricular_time > 0 and duration >= 0.5:
                activity_time = min(duration, extracurricular_time)
                day_schedule.append((start.strftime('%H:%M'), self.add_minutes(start.strftime('%H:%M'), int(activity_time * 60)), 'Extracurricular/Sports'))
                extracurricular_time -= activity_time
            elif duration >= 0.25:
                day_schedule.append((start.strftime('%H:%M'), end.strftime('%H:%M'), 'Free Time'))

    def add_minutes(self, time_str, minutes):
        time = datetime.strptime(time_str, '%H:%M')
        time += timedelta(minutes=minutes)
        return time.strftime('%H:%M')

    def display_schedule(self):
        for day, activities in self.schedule.items():
            print(f"\n{day}:")
            for start, end, activity in sorted(activities, key=lambda x: x[0]):
                print(f"  {start} - {end}: {activity}")

    def get_user_feedback(self):
        print("\nPlease provide feedback on today's schedule (1-5, where 1 is poor and 5 is excellent):")
        productivity = int(input("Overall productivity: "))
        satisfaction = int(input("Overall satisfaction: "))
        self.feedback_data.append({
            'productivity': productivity,
            'satisfaction': satisfaction,
            'study_time': self.user_preferences['self_study_time'],
            'extracurricular_time': self.user_preferences['extracurricular_time'],
            'gpa': self.user_preferences['gpa'],
            'calories_burned': self.user_preferences['calories_burned']
        })

    def update_model(self):
        if len(self.feedback_data) > 10:
            df = pd.DataFrame(self.feedback_data)
            X = df[['study_time', 'extracurricular_time', 'gpa', 'calories_burned']]
            y_prod = df['productivity']
            y_sat = df['satisfaction']
            
            prod_model = RandomForestRegressor(n_estimators=100, random_state=42)
            sat_model = RandomForestRegressor(n_estimators=100, random_state=42)
            
            prod_model.fit(X, y_prod)
            sat_model.fit(X, y_sat)
            
            # Use the models to suggest improvements
            current_state = np.array([[
                self.user_preferences['self_study_time'],
                self.user_preferences['extracurricular_time'],
                self.user_preferences['gpa'],
                self.user_preferences['calories_burned']
            ]])
            
            step = 0.25
            directions = [
                (step, 0, 0, 0),
                (-step, 0, 0, 0),
                (0, step, 0, 0),
                (0, -step, 0, 0)
            ]
            
            best_improvement = 0
            best_direction = None
            
            for direction in directions:
                new_state = current_state + direction
                prod_improvement = prod_model.predict(new_state)[0] - prod_model.predict(current_state)[0]
                sat_improvement = sat_model.predict(new_state)[0] - sat_model.predict(current_state)[0]
                total_improvement = prod_improvement + sat_improvement
                
                if total_improvement > best_improvement:
                    best_improvement = total_improvement
                    best_direction = direction
            
            if best_direction is not None:
                self.user_preferences['self_study_time'] += best_direction[0]
                self.user_preferences['extracurricular_time'] += best_direction[1]
                print("\nBased on your feedback, we've adjusted your schedule slightly for better results.")

    def add_special_event(self):
        day = input("Enter the day for the special event: ")
        start_time = input("Enter the start time (HH:MM): ")
        duration = float(input("Enter the duration in hours: "))
        event_name = input("Enter the event name: ")
        prep_time = float(input("Enter the preparation time in hours: "))
        
        end_time = self.add_minutes(start_time, int(duration * 60))
        self.schedule[day].append((start_time, end_time, event_name))
        
        # Distribute prep time
        prep_slots = self.find_free_slots(self.schedule[day])
        remaining_prep = prep_time
        for start, end in prep_slots:
            if remaining_prep <= 0:
                break
            slot_duration = (end - start).total_seconds() / 3600
            prep_duration = min(slot_duration, remaining_prep)
            prep_end = self.add_minutes(start.strftime('%H:%M'), int(prep_duration * 60))
            self.schedule[day].append((start.strftime('%H:%M'), prep_end, f"{event_name} Prep"))
            remaining_prep -= prep_duration

        self.schedule[day].sort(key=lambda x: x[0])

    def run(self):
        self.get_timetable()
        self.get_user_preferences()
        while True:
            self.generate_schedule()
            self.display_schedule()
            self.get_user_feedback()
            self.update_model()
            
            choice = input("\nDo you want to add a special event? (yes/no): ").lower()
            if choice == 'yes':
                self.add_special_event()
            
            choice = input("Do you want to generate a new schedule? (yes/no): ").lower()
            if choice != 'yes':
                break

        print("Thank you for using the Dynamic Personalized Scheduler!")

if __name__ == "__main__":
    scheduler = DynamicScheduler()
    scheduler.run()